<a href="https://colab.research.google.com/github/bmill42/streaming-data/blob/main/Merging_listening_data_and_API_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Install spotipy, import the libraries we need, and set up the credentials to get data from the Spotify API. **Make sure to insert your own `client_id` and `client_secret`.**

Then load the listening data. Again, replace the filepath with the path to your own data.

In [ ]:
!pip install Spotipy

In [ ]:
import pandas as pd
from google.colab import drive
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

drive.mount('/content/drive')

client_id = ''
client_secret = ''

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

**NOTE:** if you're using Apple Music data, skip this cell and move on to "Getting Features for Apple Music Data" below.

In [ ]:
df = pd.read_json('/content/drive/MyDrive/COMPFOR 304/Data/BAM - Streaming_History_Audio_2013-2024.json')

# Getting audio features

We'll test this on a subset of the rows using `head()` so we don't request 14k+ tracks' audio features.

The `uri_subset` variable stores a list of all the Spotify URIs for the tracks in the dataframe. Since many tracks will appear multiple times in the full dataset, we can drop duplicates here so we only request each song's features once.

In [ ]:
tracks_subset = df.head()
uri_subset = tracks_subset.spotify_track_uri.drop_duplicates()
uri_subset

Now we just ask for the audio features from the API and put them in a dataframe, exactly like in the original API examples.

In [ ]:
ft = sp.audio_features(uri_subset)
ft_df = pd.DataFrame(ft)
ft_df

# Merging audio features with listening data

Now we **merge** the features with the listening data by asking pandas to match up the URIs in our original data with the URIs in the audio features that we downloaded.

In [ ]:
subset_merged = pd.merge(tracks_subset, ft_df, left_on='spotify_track_uri', right_on='uri')

Looking at the columns of the resulting table, we can see that the original columns from the listening data (timestamp, track and artist name, and so on) have been augmented with the audio features, like 'danceability' and 'speechiness'.

In [ ]:
subset_merged.columns

There are still a lot of unneccessary columns here, so it's typically going to be best to reduce the table down to just the ones we care about.

**It would be a great idea to export the data as a CSV file now and save it to your computer and Google Drive so you don't have to request the features again every time!**

In [ ]:
subset_merged[['ts', 'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'spotify_track_uri', 'reason_end', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms']]

# Getting audio features for Apple Music data

Load the Apple Music data from a CSV file and split the 'Track Description' column into separate 'Artist' and 'Track Name' columns.

**The rest of this code should run as-is once you enter the correct file path here.**

In [ ]:
apple_df = pd.read_csv('/content/drive/MyDrive/COMPFOR 304/Data/apple_music_test.csv')

apple_df[['Artist', 'Track Name']] = apple_df['Track Description'].str.split(" - ", expand=True, n=1)
apple_df['spotify_uri'] = ''

Now, we need to go through the tracks in the Apple Music data one by one and collect their Spotify URIs, which will then allow us to collect the audio features just like in the above example for the native Spotify data.

We'll do this by looping over every track and using the Spotify API to do a *search* - essentially the same as typing a track and artist name into the search bar in the Spotify app. We will select the first track that comes up in the search and grab its URI.

This is an imperfect approach since a search is not guaranteed to turn up the right track for multiple reasons: an artist may have recorded the same song multiple times, e.g. on both live and studio albums; a track may be on Apple Music but not Spotify; or metadata issues may make the actual artist of a track ambiguous, as will probably happen often with classical music.

In these kinds of cases, the top result for our search will likely be the wrong track, and we'll end up with the wrong URI - in some cases potentially for a track from an entirely different artist. This will degrade the quality of our data, but if the number of problematic tracks is small enough it likely won't affect the quality of any resulting analysis too much.

In [ ]:
apple_dict = apple_df[['Artist', 'Track Name']].drop_duplicates().to_dict(orient='records')

for r in apple_dict:
    search_results = sp.search(q='track:' + r['Track Name'] + ' artist:' + r['Artist'], type='track')
    best_uri = search_results['tracks']['items'][0]['uri']
    r['uri'] = best_uri

Now we just turn the dictionary containing the track and artist names and URIs into a dataframe and merge it back into the original Apple Music data.

In [ ]:
uri_df = pd.DataFrame(apple_dict)
apple_merged = pd.merge(apple_df, uri_df, on=['Artist', 'Track Name'])

From here, the process is identical to the Spotify example above: isolate the list of URIs, request the audio features via the API, turn the results into a dataframe and merge it back into the full dataset.

In [ ]:
apple_uris = apple_merged.uri

apple_ft = sp.audio_features(apple_uris)
apple_ft_df = pd.DataFrame(apple_ft)

In [ ]:
apple_final = pd.merge(apple_merged, apple_ft_df, on='uri')

The final merged dataset still contains some of the extraneous columns that came with the API call, like some URLs that are not useful for our purposes; at this point it would make sense to clean up the full table before continuing to work with it.

**It would be a great idea to export the data as a CSV file now and save it to your computer and Google Drive so you don't have to request the features again every time!**

In [ ]:
apple_final